# Vektordatenbank Suche
Ein Notebook nur spezialisiert auf die Suche in der Vektordatenbank

In [8]:
from classes import get_test_data, MitarbeiterSkills, TechnologieWissen
from embeddings_utils import get_or_create_embeddings, get_query_embedding
from visualisation import plot_with_query_umap
import numpy as np
import faiss
import ipywidgets as widgets
from IPython.display import display, clear_output

# Testdaten laden und ggf. erweitern
workers = get_test_data()

# Zusätzliche Test-Mitarbeiter
extra_workers = [
    MitarbeiterSkills(
        name="Boris Backend",
        technologien=[
            TechnologieWissen(name="Java", letzte_verwendung=0),
            TechnologieWissen(name="Docker", letzte_verwendung=0),
            TechnologieWissen(name="SQL", letzte_verwendung=0),
            TechnologieWissen(name="MongoDB", letzte_verwendung=0),
            TechnologieWissen(name="PostgreSQL", letzte_verwendung=0),
            TechnologieWissen(name="MySQL", letzte_verwendung=0)
        ]
    ),
    MitarbeiterSkills(
        name="Fritz Frontend",
        technologien=[
            TechnologieWissen(name="JavaScript", letzte_verwendung=0),
            TechnologieWissen(name="TypeScript", letzte_verwendung=0),
            TechnologieWissen(name="React", letzte_verwendung=0),
            TechnologieWissen(name="Vue.js", letzte_verwendung=0),
            TechnologieWissen(name="Angular", letzte_verwendung=0)
        ]
    ),
    MitarbeiterSkills(
        name="Tim Technik",
        technologien=[
            TechnologieWissen(name="Azure", letzte_verwendung=0),
            TechnologieWissen(name="AWS", letzte_verwendung=0),
            TechnologieWissen(name="GCP", letzte_verwendung=0),
            TechnologieWissen(name="Kubernetes", letzte_verwendung=0),
            TechnologieWissen(name="Docker", letzte_verwendung=0)
        ]
    )
]
workers.extend(extra_workers)

In [9]:
# OpenAI Embedding Funktion (API-Key muss konfiguriert sein)
from openai import OpenAI
client = OpenAI()
def embedd(text):
    return client.embeddings.create(input=text, model="text-embedding-3-small").data[0].embedding

In [10]:
# Embeddings laden oder erzeugen
embedded_data = get_or_create_embeddings(workers, embedd)

Lade gespeicherte Embeddings...


In [11]:
# FAISS Index initialisieren
if len(embedded_data.shape) == 1:
    embedded_data = np.expand_dims(embedded_data, axis=0)
dim = embedded_data.shape[1]
index = faiss.IndexFlatL2(dim)
embeddings_np = embedded_data.astype("float32")
index.add(embeddings_np)

## Interaktive Suche
Geben Sie eine Suchanfrage ein, um ähnliche Mitarbeiter zu finden und die Ergebnisse zu visualisieren.

In [12]:
def search_and_visualize(query, erg_anzahl=5):
    query_vector = get_query_embedding(query, embedd)
    search_vector = np.array([query_vector]).astype('float32')
    D, indices = index.search(search_vector, erg_anzahl)
    hover_texts = [str(w) for w in workers]
    plot_with_query_umap(embeddings_np, query_vector, highlight_indices=indices[0], hover_texts=hover_texts)
    print(f"Distances: {D}")
    print(f"Indices: {indices}")
    for i in indices[0]:
        print(workers[i])

In [13]:
query_input = widgets.Text(
    value='Erfahrung mit Backend Technologien',
    placeholder='Suchanfrage eingeben...',
    description='Query:',
    disabled=False
)
output = widgets.Output()

def on_query_change(change):
    with output:
        clear_output(wait=True)
        search_and_visualize(change['new'])

query_input.observe(on_query_change, names='value')
display(query_input, output)
# Initial plot
on_query_change({'new': query_input.value})

Text(value='Erfahrung mit Backend Technologien', description='Query:', placeholder='Suchanfrage eingeben...')

Output()

## Notitz
Es gibt 2 Ansätze, die man noch probieren könnte.

1. Alle Daten vor dem Embedding in Fließtext konvertieren
2. Query-Vektor in Stichpunkte bzw. konkrete Technologie suche Konvertieren (siehe Excalidraw) 